# Coding Exercise

You must correctly implement the function described in the prompt below.

Feel free to test out pieces of code to help you write the solution.

Please thoroughly test that the final code implements the function correctly.

## Prompt

**Function signature:** `minimumEnergyCost(xc: List[int], yc: List[int], xb: List[int], yb: List[int], xp: List[int], yp: List[int], esr: int) -> int`

    The celebrated general Archibald Waving took charge of the fourth army in the occidental front. After losing the first three armies, Waving has become obsessed with efficient energy consumption. He has thus started innovating in what he calls "green warfare". He will use his newly acquired philosophy in an attempt to win the final battle of the occidental front. The development of death rays has been useful for Waving's side, but the plutonium required to shoot is scarce. The enemy side has bases and energy plants of its own and thus Waving has decided to take the initiative and destroy the enemy's forces once and for all, but using as little energy as possible.

    The general has assigned you to determine how to accomplish this feat. Your main objective is to neutralize each of the enemy bases. A base is considered neutralized either if it was destroyed by a death ray or if all the energy plants supplying to it have been destroyed. There are a number of death ray canons that you can use and each canon may shoot multiple targets if necessary but can only destroy a single target with each shoot. The positions of the canons are given by two List[int]s: canonX and canonY where the i-th canon is located at the point (canonX[i],canonY[i]). You can make each canon shoot a certain target at any distance but the energy required per target is equal to: d*d gigajoules where d is the euclidean distance between the target and the canon. The bases' positions are described by List[int]s baseX and baseY where the i-th base is located at the point (baseX[i],baseY[i]), and the energy plants' locations are described by List[int]s plantX and plantY where the i-th energy plant is located at the point (plantX[i],plantY[i]). An energy plant supplies energy to an enemy base if and only if the euclidean distance between them is less than or equal to energySupplyRadius. Return the minimum amount of energy required (in gigajoules), to neutralize all enemy bases.

    Constraints
    -baseX, baseY, canonX, canonY, plantX and plantY will each contain between 1 and 50 elements, inclusive.
    -baseX and baseY will contain the same number of elements.
    -canonX and canonY will contain the same number of elements.
    -plantX and plantY will contain the same number of elements.
    -The position given for each canon, base and energy plant will be unique.
    -Each element in baseX, baseY, canonX, canonY, plantX and plantY will be between -500 and 500, inclusive.
    -energySupplyRadius will be between 1 and 2000, inclusive.
    -Every base will be within energySupplyRadius distance units to at least one energy plant.
 
    Examples
    0)
        { 0 }
    { 0 }
    {1,2,3}
    {0,0,0}
    {3}
    {3}
    4

    Returns: 14
    Use the canon to destroy each base individually, the costs are: 1, 4 and 9 gigajoules. The total cost is 14 gigajoules.

    1)
        { 0 }
    { 0 }
    {1,2,3}
    {0,0,0}
    {2}
    {2}
    4

    Returns: 8
    In this case, the distance between the energy plant and the canon is smaller and the new optimal strategy is to destroy the energy plant with a cost of 8 gigajoules.

    2)
        {3,6}
    {3,6}
    {1,2,3,4,5}
    {5,4,2,3,1}
    {1,2,5}
    {1,2,5}
    5

    Returns: 12
    Use the first canon to destroy the first two energy plants, use the second canon to destroy the remaining one.

    3)
        {0}
    {0}
    {-10,-10,10}
    {10,-10,0}
    {10,10,-10}
    {10,-10,0}
    10

    Returns: 200
    Destroy the base at (10,0) and the energy plant at (-10,0), 200 gigajoules are required in total.

    4)
        {0}
    {0}
    {3}
    {3}
    {1,2,3}
    {0,0,0}
    4

    Returns: 14

    

I spend a lot of time thinking.

The idea is to take down each of the bases recursively.
At each step we will take down a base by either destroing the base or the power plants.
Destruction of power plants take priority, therefore I will always take down the power plants if the cost is smaller than the cost to take the bases down.
If the destruction of the power plants has a higher cost than the cost to take the base down, we need to try take down both.
After taking a base down, we will call the function recursively with either one less base or less power plants.

What is the problem with this algorithm?
The worst case scenario is the case when we need to try all possible combinations:
there are up to 50 bases, and at each step we can destroy the bases or the power plants ($2^{50}$).

There is the possibility to use dynamic programming, but I think there are many different combinations to keep track of, therefore I think we will run out of memory.

Another possible solution is to keep track of the minimal cost calculated and return early if the cost exceeds the current minimal cost.

First, let me write a function to calculate the cost to remove a target.
The function will find the closest cannon to shoot the base, and the power plants

In [18]:
from numpy import array
from typing import List


def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            
    return cost

xc = array([0,1,2,3,4])
yc = array([0,2,3,4,5])

print(calculate_cost(xc,yc,xc,yc,xc,yc,1,1))

{'base': 0, 'power_idx': [1], 'power': 0}


In [19]:
print(calculate_cost(xc,yc,xc,yc,xc,yc,5,1))
print('should be 5')

{'base': 0, 'power_idx': [0, 1, 2], 'power': 0}
should be 5


Looks like the function is working.

Now I will implement a function to take down iteractively

In [20]:
import copy
from numpy import inf

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    print(cost)
    
    #always better to take down power plants
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    #else take down base
    else:
        
        total_cost += cost['base']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = copy(xp)
        new_yp = copy(yp)
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    return total_cost

take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)

{'base': 1, 'power_idx': [], 'power': 0}
{'base': 4, 'power_idx': [], 'power': 0}
{'base': 9, 'power_idx': [], 'power': 0}


0

0 is the wrong result.

After some investigation I realized that, in calculate_cost, I need to compare to the radius square.

In [22]:
def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr*esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            print(xt, yt, cost['power'])
            
    return cost

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    print(cost)
    
    #take down power plants
    #always better to take down power plants if cost is smaller
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    #else take down base
    else:
        
        total_cost += cost['base']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = copy.copy(xp)
        new_yp = copy.copy(yp)
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    return total_cost

take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)

1 0 18
{'base': 1, 'power_idx': [0], 'power': 18}


TypeError: 'module' object is not callable

I should call copy.copy 

In [25]:
def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr*esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            print(xt, yt, cost['power'])
            
    return cost

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    print(cost)
    
    #take down power plants
    #always better to take down power plants if cost is smaller
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    #else take down base
    else:
        
        total_cost += cost['base']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = copy.copy(xp)
        new_yp = copy.copy(yp)
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    return total_cost

assert (a := take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)) == 14, a

1 0 18
{'base': 1, 'power_idx': [0], 'power': 18}
2 0 18
{'base': 4, 'power_idx': [0], 'power': 18}
3 0 18
{'base': 9, 'power_idx': [0], 'power': 18}


In [26]:
assert (a := take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[2],[2],4)) == 8, a

1 0 8
{'base': 1, 'power_idx': [0], 'power': 8}
2 0 8
{'base': 4, 'power_idx': [0], 'power': 8}
3 0 8
{'base': 9, 'power_idx': [0], 'power': 8}


AssertionError: 13

The algorithm is not doing what was discussed in the beginning.

When cost['base'] < cost['power'], I need to try to remove both the base and the power plants.

In [28]:
from numpy import isnan

def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr*esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            print(xt, yt, cost['power'])
            
    return cost

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    print(cost)
    
    #take down power plants
    #always better to take down power plants if cost is smaller
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    #try to takedown power plants or bases
    else:
        
        #take down power plant
        cost_p = total_cost + cost['power']
        
        if cost_p >= min_cost:
            cost_p = inf
        
        else:
            new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
            new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_p = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_p)
        
        #take down base
        cost_b = total_cost + cost['base']
        
        if cost_b >= min_cost:
            cost_b = inf
        
        else:
            new_xp = copy.copy(xp)
            new_yp = copy.copy(yp)
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_b = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_b)
            
        if isnan(cost_p) and isnan(cost_b):
            return inf
        elif cost_p <= cost_b:
            total_cost = cost_p
        else:
            total_cost = cost_b
        
    return total_cost

assert (a := take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)) == 14, a

1 0 18
{'base': 1, 'power_idx': [0], 'power': 18}
2 0 18
{'base': 4, 'power_idx': [0], 'power': 18}
3 0 18
{'base': 9, 'power_idx': [0], 'power': 18}


In [29]:
assert (a := take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[2],[2],4)) == 8, a

1 0 8
{'base': 1, 'power_idx': [0], 'power': 8}
2 0 8
{'base': 4, 'power_idx': [0], 'power': 8}
3 0 8
{'base': 9, 'power_idx': [0], 'power': 8}


In [31]:
assert(a:=take_down([3,6],[3,6],[1,2,3,4,5],[5,4,2,3,1],[1,2,5],[1,2,5],5))==12, a

1 5 8
1 5 10
1 5 12
{'base': 8, 'power_idx': [0, 1, 2], 'power': 12}
2 4 8
2 4 10
2 4 12
{'base': 2, 'power_idx': [0, 1, 2], 'power': 12}
3 2 8
3 2 10
3 2 12
{'base': 1, 'power_idx': [0, 1, 2], 'power': 12}
4 3 8
4 3 10
4 3 12
{'base': 1, 'power_idx': [0, 1, 2], 'power': 12}
5 1 8
5 1 10
5 1 12
{'base': 8, 'power_idx': [0, 1, 2], 'power': 12}


In [33]:
assert (a:=take_down([0],[0],[-10,-10,10],[10,-10,0],[10,10,-10],[10,-10,0],10)) == 200, a

-10 10 100
{'base': 200, 'power_idx': [2], 'power': 100}
{'base': 200, 'power_idx': [], 'power': 0}
10 0 200
10 0 400
{'base': 100, 'power_idx': [0, 1], 'power': 400}


In [34]:
assert (a:=take_down([0],[0],[3],[3],[1,2,3],[0,0,0],4))==14, a

3 3 1
3 3 5
3 3 14
{'base': 18, 'power_idx': [0, 1, 2], 'power': 14}


All the results are correct.

Let me remove the prints and organize the assertions

In [36]:
def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr*esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            
    return cost

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    
    #take down power plants
    #always better to take down power plants if cost is smaller
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    #try to takedown power plants or bases
    else:
        
        #take down power plant
        cost_p = total_cost + cost['power']
        
        if cost_p >= min_cost:
            cost_p = inf
        
        else:
            new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
            new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_p = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_p)
        
        #take down base
        cost_b = total_cost + cost['base']
        
        if cost_b >= min_cost:
            cost_b = inf
        
        else:
            new_xp = copy.copy(xp)
            new_yp = copy.copy(yp)
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_b = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_b)
            
        if isnan(cost_p) and isnan(cost_b):
            return inf
        elif cost_p <= cost_b:
            total_cost = cost_p
        else:
            total_cost = cost_b
        
    return total_cost

def test():
    assert (a := take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)) == 14, a
    assert (a := take_down([ 0 ],[ 0 ],[1,2,3],[0,0,0],[2],[2],4)) == 8, a
    assert (a:=take_down([3,6],[3,6],[1,2,3,4,5],[5,4,2,3,1],[1,2,5],[1,2,5],5))==12, a
    assert (a:=take_down([0],[0],[-10,-10,10],[10,-10,0],[10,10,-10],[10,-10,0],10)) == 200, a
    assert (a:=take_down([0],[0],[3],[3],[1,2,3],[0,0,0],4))==14, a
    print('everything is correct')
    
test()

everything is correct


Lets test the biggest problem of this problem: performance. 

In [41]:
from datetime import datetime

size = 50
radius = 100000000000

xc, yc, xp, yp = [], [], [], []

#50 power plants at (1,1), (2,2), (3,3)...
for i in range(1,size+1):
    xp.append(i)
    yp.append(i)
    
#50 cannons at (0,1),(0,2), (0,3)...
for i in range(1, size+1):
    xc.append(0)
    yc.append(i)

for s in range(1,size+1):
    
    xb, yb = [], []
    
    #bases at (1,1),(1,2),(1,3)...
    for i in range(1,size+1):
        xb.append(1)
        yb.append(i)
    
    
    t1 = datetime.now()
    
    res = take_down(xc, yc, xb, yb, xp, yp, radius)
    
    t2 = datetime.now()
    
    #res should be have value i
    print(s, t2- t1, res, i)

1 0:00:00.115158 50 50
2 0:00:00.114159 50 50
3 0:00:00.114845 50 50
4 0:00:00.113962 50 50
5 0:00:00.114635 50 50
6 0:00:00.115611 50 50
7 0:00:00.114630 50 50
8 0:00:00.112990 50 50
9 0:00:00.113653 50 50
10 0:00:00.113500 50 50
11 0:00:00.113316 50 50
12 0:00:00.119268 50 50
13 0:00:00.114159 50 50
14 0:00:00.113176 50 50
15 0:00:00.113755 50 50
16 0:00:00.113747 50 50
17 0:00:00.114804 50 50
18 0:00:00.115251 50 50
19 0:00:00.114822 50 50
20 0:00:00.115475 50 50
21 0:00:00.114091 50 50
22 0:00:00.113399 50 50
23 0:00:00.113590 50 50
24 0:00:00.113354 50 50
25 0:00:00.113945 50 50
26 0:00:00.113874 50 50
27 0:00:00.114171 50 50
28 0:00:00.113115 50 50
29 0:00:00.114114 50 50
30 0:00:00.113476 50 50
31 0:00:00.113793 50 50
32 0:00:00.113492 50 50
33 0:00:00.113365 50 50
34 0:00:00.114218 50 50
35 0:00:00.115527 50 50
36 0:00:00.115045 50 50
37 0:00:00.114395 50 50
38 0:00:00.113751 50 50
39 0:00:00.114143 50 50
40 0:00:00.113760 50 50
41 0:00:00.114036 50 50
42 0:00:00.112909 50 50
4

I am not testing properly. The for inside should run up to s.
Also, now I will randomize the order of the bases.
We could have generated the bases in a order that benefits our algorithm.

In [45]:
import random

size = 50
radius = 100000000000

xc, yc, xp, yp = [], [], [], []

#50 power plants at (1,1), (2,2), (3,3)...
for i in range(1,size+1):
    xp.append(i)
    yp.append(i)
    
#50 cannons at (0,1),(0,2), (0,3)...
for i in range(1, size+1):
    xc.append(0)
    yc.append(i)

for s in range(1,size+1):
    
    xb, yb = [], []
    
    #bases at (1,1),(1,2),(1,3)...
    for i in range(1,s+1):
        xb.append(1)
        yb.append(i)
    
    #shuffle xb, yb
    temp = list(zip(xb, yb))
    random.shuffle(temp)
    xb, yb = zip(*temp)
    
    t1 = datetime.now()
    
    res = take_down(xc, yc, xb, yb, xp, yp, radius)
    
    t2 = datetime.now()
    
    #res should be have value i
    print(s, t2- t1, res, s)

1 0:00:00.002648 1 1
2 0:00:00.004992 2 2
3 0:00:00.007133 3 3
4 0:00:00.009012 4 4
5 0:00:00.011274 5 5
6 0:00:00.013803 6 6
7 0:00:00.015997 7 7
8 0:00:00.018080 8 8
9 0:00:00.020390 9 9
10 0:00:00.022912 10 10
11 0:00:00.025327 11 11
12 0:00:00.026980 12 12
13 0:00:00.029193 13 13
14 0:00:00.032752 14 14
15 0:00:00.034501 15 15
16 0:00:00.036477 16 16
17 0:00:00.038920 17 17
18 0:00:00.041272 18 18
19 0:00:00.043385 19 19
20 0:00:00.046808 20 20
21 0:00:00.048928 21 21
22 0:00:00.050246 22 22
23 0:00:00.053045 23 23
24 0:00:00.055127 24 24
25 0:00:00.058154 25 25
26 0:00:00.059214 26 26
27 0:00:00.061227 27 27
28 0:00:00.063428 28 28
29 0:00:00.067319 29 29
30 0:00:00.068120 30 30
31 0:00:00.070652 31 31
32 0:00:00.072853 32 32
33 0:00:00.075884 33 33
34 0:00:00.077147 34 34
35 0:00:00.083316 35 35
36 0:00:00.083134 36 36
37 0:00:00.084023 37 37
38 0:00:00.087170 38 38
39 0:00:00.088282 39 39
40 0:00:00.090933 40 40
41 0:00:00.093426 41 41
42 0:00:00.094988 42 42
43 0:00:00.109558 4

Execution time is looking fine.

Let me change reorganize so that the final function has the same name as the prompt 

In [47]:
def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr*esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            
    return cost

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    
    #take down power plants
    #always better to take down power plants if cost is smaller
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost)
        
    #try to takedown power plants or bases
    else:
        
        #take down power plant
        cost_p = total_cost + cost['power']
        
        if cost_p >= min_cost:
            cost_p = inf
        
        else:
            new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
            new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_p = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_p)
        
        #take down base
        cost_b = total_cost + cost['base']
        
        if cost_b >= min_cost:
            cost_b = inf
        
        else:
            new_xp = copy.copy(xp)
            new_yp = copy.copy(yp)
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_b = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_b)
            
        if isnan(cost_p) and isnan(cost_b):
            return inf
        elif cost_p <= cost_b:
            total_cost = cost_p
        else:
            total_cost = cost_b
        
    return total_cost

def minimumEnergyCost(xc: List[int], yc: List[int], xb: List[int], yb: List[int], xp: List[int], yp: List[int], esr: int) -> int:
    return take_down(xc, yc, xb, yb, xp, yp, esr)

def test():
    assert (a := minimumEnergyCost([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)) == 14, a
    assert (a := minimumEnergyCost([ 0 ],[ 0 ],[1,2,3],[0,0,0],[2],[2],4)) == 8, a
    assert (a:=minimumEnergyCost([3,6],[3,6],[1,2,3,4,5],[5,4,2,3,1],[1,2,5],[1,2,5],5))==12, a
    assert (a:=minimumEnergyCost([0],[0],[-10,-10,10],[10,-10,0],[10,10,-10],[10,-10,0],10)) == 200, a
    assert (a:=minimumEnergyCost([0],[0],[3],[3],[1,2,3],[0,0,0],4))==14, a
    print('everything is correct')
    
test()

everything is correct


I miss one edge case:
The case where it is better to remove bases, and the power plants are positioned such that the i-th base always share a power plant with the (i+1)-th base.

In [49]:
import random

size = 50
radius = 4

xc, yc, xp, yp = [], [], [], []

#50 power plants at (1,1), (2,2), (3,3)...
for i in range(1,size+1):
    xp.append(-1)
    yp.append(i)
    
#50 cannons at (0,1),(0,2), (0,3)...
for i in range(1, size+1):
    xc.append(0)
    yc.append(i)

for s in range(1,size+1):
    
    xb, yb = [], []
    
    #bases at (1,1),(1,2),(1,3)...
    for i in range(1,s+1):
        xb.append(1)
        yb.append(i)
    
    #shuffle xb, yb
    temp = list(zip(xb, yb))
    random.shuffle(temp)
    xb, yb = zip(*temp)
    
    t1 = datetime.now()
    
    res = minimumEnergyCost(xc, yc, xb, yb, xp, yp, radius)
    
    t2 = datetime.now()
    
    #res should be have value i
    print(s, t2- t1, res, s)

1 0:00:00.000321 1 1
2 0:00:00.000751 2 2
3 0:00:00.001406 3 3
4 0:00:00.002466 4 4
5 0:00:00.003537 5 5
6 0:00:00.004916 6 6
7 0:00:00.006457 7 7
8 0:00:00.014125 8 8
9 0:00:00.023501 9 9
10 0:00:00.027962 10 10
11 0:00:00.038907 11 11
12 0:00:00.051681 12 12
13 0:00:00.097460 13 13
14 0:00:00.135164 14 14
15 0:00:00.210615 15 15
16 0:00:00.311707 16 16
17 0:00:00.444376 17 17
18 0:00:00.586490 18 18
19 0:00:00.952293 19 19
20 0:00:01.643161 20 20
21 0:00:02.153821 21 21
22 0:00:03.249645 22 22
23 0:00:03.972531 23 23
24 0:00:06.873058 24 24
25 0:00:08.394587 25 25
26 0:00:10.246362 26 26
27 0:00:17.392759 27 27
28 0:00:31.212617 28 28
29 0:00:36.943983 29 29
30 0:01:01.321531 30 30
31 0:00:52.008866 31 31
32 0:02:12.149230 32 32


KeyboardInterrupt: 

Execution time is bad.
Let me try to improve the solution by adding a memory.

The memory will keep track of the possible states as a hash and save the results.

In [54]:
def calculate_cost(xc: List[int], yc: List[int],
                   xb: List[int], yb: List[int],
                   xp: List[int], yp: List[int],
                   esr: int, target: int):
    #calculate the cost to take target down
    #can take both: the base and the power plants
    cost = {}
    
    xt, yt = xb[target], yb[target]
    
    cost['base'] = min([
        (x-xt)**2+(y-yt)**2
        for x,y in zip(xc,yc)
    ])
    
    cost['power_idx'] = []
    cost['power'] = 0
    
    for i,(x,y) in enumerate(zip(xp,yp)):
        
        #if power plant supplies the target base
        if (xt-x)**2 + (yt-y)**2 <= esr*esr:
            cost['power'] += min([
                (x-xx)**2+(y-yy)**2
                for xx,yy in zip(xc,yc)
            ])
            cost['power_idx'].append(i)
            
    return cost

def take_down(xc: List[int], yc: List[int],
              xb: List[int], yb: List[int],
              xp: List[int], yp: List[int],
              esr: int, min_cost=inf, total_cost=0, memory=None):
    
    #if everyone is down the cost is zero
    if len(xb) == 0 or len(xp) == 0:
        return total_cost
    
    #
    if memory is None:
        memory = dict()
        
    h = hash(str([xc,yc, xb, yb, xp, yp]))
    if h in memory.keys():
        return memory[h]
    
    cost = calculate_cost(xc,yc, xb, yb, xp, yp, esr, target=0)
    
    #take down power plants
    #always better to take down power plants if cost is smaller
    if cost['power'] <= cost['base']:
        
        total_cost += cost['power']
        
        if total_cost >= min_cost:
            return inf
        
        new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
        new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
        new_xb = xb[1:]
        new_yb = yb[1:]
        
        total_cost = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, total_cost, memory)
        
    #try to takedown power plants or bases
    else:
        
        #take down power plant
        cost_p = total_cost + cost['power']
        
        if cost_p >= min_cost:
            cost_p = inf
        
        else:
            new_xp = [x for i,x in enumerate(xp) if i not in cost['power_idx']]
            new_yp = [y for i,y in enumerate(yp) if i not in cost['power_idx']]
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_p = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_p, memory)
        
        #take down base
        cost_b = total_cost + cost['base']
        
        if cost_b >= min_cost:
            cost_b = inf
        
        else:
            new_xp = copy.copy(xp)
            new_yp = copy.copy(yp)
            new_xb = xb[1:]
            new_yb = yb[1:]

            cost_b = take_down(xc, yc, new_xb, new_yb, new_xp, new_yp, esr, min_cost, cost_b, memory)
            
        if isnan(cost_p) and isnan(cost_b):
            return inf
        elif cost_p <= cost_b:
            total_cost = cost_p
        else:
            total_cost = cost_b
            
    memory[h] = total_cost
        
    return total_cost

def minimumEnergyCost(xc: List[int], yc: List[int], xb: List[int], yb: List[int], xp: List[int], yp: List[int], esr: int) -> int:
    return take_down(xc, yc, xb, yb, xp, yp, esr)

def test():
    assert (a := minimumEnergyCost([ 0 ],[ 0 ],[1,2,3],[0,0,0],[3],[3],4)) == 14, a
    assert (a := minimumEnergyCost([ 0 ],[ 0 ],[1,2,3],[0,0,0],[2],[2],4)) == 8, a
    assert (a:=minimumEnergyCost([3,6],[3,6],[1,2,3,4,5],[5,4,2,3,1],[1,2,5],[1,2,5],5))==12, a
    assert (a:=minimumEnergyCost([0],[0],[-10,-10,10],[10,-10,0],[10,10,-10],[10,-10,0],10)) == 200, a
    assert (a:=minimumEnergyCost([0],[0],[3],[3],[1,2,3],[0,0,0],4))==14, a
    print('everything is correct')
    
test()

everything is correct


In [55]:
size = 50
radius = 4

xc, yc, xp, yp = [], [], [], []

#50 power plants at (1,1), (2,2), (3,3)...
for i in range(1,size+1):
    xp.append(-1)
    yp.append(i)
    
#50 cannons at (0,1),(0,2), (0,3)...
for i in range(1, size+1):
    xc.append(0)
    yc.append(i)

for s in range(1,size+1):
    
    xb, yb = [], []
    
    #bases at (1,1),(1,2),(1,3)...
    for i in range(1,s+1):
        xb.append(1)
        yb.append(i)
    
    #shuffle xb, yb
    temp = list(zip(xb, yb))
    random.shuffle(temp)
    xb, yb = zip(*temp)
    
    t1 = datetime.now()
    
    res = minimumEnergyCost(xc, yc, xb, yb, xp, yp, radius)
    
    t2 = datetime.now()
    
    #res should be have value i
    print(s, t2- t1, res, s)

1 0:00:00.000348 1 1
2 0:00:00.000821 2 2
3 0:00:00.001546 3 3
4 0:00:00.002392 4 4
5 0:00:00.003598 5 5
6 0:00:00.005507 6 6
7 0:00:00.007123 7 7
8 0:00:00.009771 8 8
9 0:00:00.012288 9 9
10 0:00:00.019889 10 10
11 0:00:00.025587 11 11
12 0:00:00.032961 12 12
13 0:00:00.039242 13 13
14 0:00:00.062945 14 14
15 0:00:00.080968 15 15
16 0:00:00.090535 16 16
17 0:00:00.115891 17 17
18 0:00:00.133454 18 18
19 0:00:00.184843 19 19
20 0:00:00.289110 20 20
21 0:00:00.379630 21 21
22 0:00:00.458783 22 22
23 0:00:00.486891 23 23
24 0:00:00.661658 24 24
25 0:00:01.016419 25 25
26 0:00:01.124409 26 26
27 0:00:01.565622 27 27
28 0:00:01.479170 28 28
29 0:00:02.770399 29 29
30 0:00:03.758906 30 30
31 0:00:04.297214 31 31
32 0:00:06.379877 32 32
33 0:00:06.497949 33 33
34 0:00:09.657864 34 34
35 0:00:11.249988 35 35
36 0:00:13.314066 36 36
37 0:00:17.617739 37 37
38 0:00:20.875944 38 38
39 0:00:21.685450 39 39
40 0:00:32.845448 40 40
41 0:00:37.622630 41 41
42 0:00:51.736296 42 42
43 0:01:16.160800 4

It is very slow for the largest problems, but is solves the problem.